### This is from a Kaggle competition: https://www.kaggle.com/c/nlp-getting-started/data?select=train.csv

In [1]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
df_train = df_train[:10]

In [3]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
import sidetable
df_train.stb.missing(style=True)

,missing,total,percent
keyword,10,10,100.00%
location,10,10,100.00%
id,0,10,0.00%
text,0,10,0.00%
target,0,10,0.00%


In [5]:
len(max(df_train.text))

79

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np
def get_pad_sq(df):
    text = np.array(df.text)
    tokenizer = Tokenizer(num_words=len(df.text.unique()))
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    # Find the max length of rows in sequences
    max_length = 0
    for sentence in sequences:
        max_length = max(max_length, len(sentence))
    df = sequence.pad_sequences(sequences, maxlen=max_length)
    return (df, max_length)

In [7]:
test_id = df_test.id

In [8]:
train, _ = get_pad_sq(df_train)
test, max_review_length = get_pad_sq(df_test)
train_target = np.array(df_train.target)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, train_target, stratify=train_target, test_size=0.2, random_state=1)

Credit : https://medium.com/@mrunal68/text-sentiments-classification-with-cnn-and-lstm-f92652bc29fd

In [13]:
# Credit source: https://github.com/lukenew2/hyper_parameter_optimization/blob/master/hyper_parameter_optimization.ipynb

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense,Conv1D,MaxPooling1D
from tensorflow.keras import regularizers
from keras.layers import LSTM,Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
def build_model(input_length, dropout2_rate=0.5):
    # create the model
    embedding_vector_length = 128
    top_words = 10000
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=input_length, name='embed_1'))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', name='conv1d_1'))
    model.add(MaxPooling1D(pool_size=2, name='maxpool1d_1'))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, name='lstm_1'))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), name='dense_1'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#     checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy',verbose=0, save_best_only=True)
#     es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=1)
#     callbacks_list = [checkpoint, es]
#     model.fit(X_train, y_train, epochs=4000, batch_size=32,verbose = 1, callbacks = callbacks_list, validation_data=(X_val,y_val))

In [14]:
import keras
keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)